In [ ]:
import os

# Ruta de la carpeta original
base_drive_path = "G:/Mi unidad/TESIS"  # Asegurar que esta ruta es correcta
data_directory = os.path.join(base_drive_path, "MURA_UCD")

def count_images(directory):
    """Cuenta todas las imágenes dentro del directorio dado."""
    total_images = 0
    for root, _, files in os.walk(directory):
        total_images += sum(1 for file in files if file.endswith(('.png', '.jpg', '.jpeg')))
    return total_images

# Contar imágenes en MURA-v1.1
image_count = count_images(data_directory)
print(f"🖼 Total de imágenes en {data_directory}: {image_count}")

🖼 Total de imágenes en G:/Mi unidad/TESIS\MURA_UCD: 40005


In [ ]:
import os
import pandas as pd

# Ruta base donde están los CSVs
ruta_base = "G:/Mi unidad/TESIS/MURA_UCD"
csv_train = os.path.join(ruta_base, "train.csv")
csv_valid = os.path.join(ruta_base, "valid.csv")

# Mapear nombres de carpetas a categorías legibles
categorias_mapa = {
    "XR_ELBOW": "Codo",
    "XR_FINGER": "Dedo",
    "XR_FOREARM": "Antebrazo",
    "XR_HAND": "Mano",
    "XR_HUMERUS": "Húmero",
    "XR_SHOULDER": "Hombro",
    "XR_WRIST": "Muñeca"
}

# Leer CSVs
df_train = pd.read_csv(csv_train, names=["ruta", "etiqueta"], skiprows=1)
df_valid = pd.read_csv(csv_valid, names=["ruta", "etiqueta"], skiprows=1)

# Agregar columna de tipo de dataset
df_train["split"] = "Train"
df_valid["split"] = "Valid"

# Unir ambos
df = pd.concat([df_train, df_valid], ignore_index=True)

# Extraer la categoría cruda desde la ruta (posición 5 del path)
df["categoria_raw"] = df["ruta"].apply(lambda x: os.path.normpath(x).split(os.sep)[5])



# Mapear a nombres legibles
df["Categoría"] = df["categoria_raw"].map(categorias_mapa)

# Asegurar que 'etiqueta' es entero
df["etiqueta"] = df["etiqueta"].astype(int)

# Inicializar el diccionario como en tu código original
conteo = {
    "Categoría": [],
    "Normal_Train": [],
    "Anormal_Train": [],
    "Normal_Valid": [],
    "Anormal_Valid": []
}

# Recorremos cada categoría en orden
for key, nombre_categoria in categorias_mapa.items():
    conteo["Categoría"].append(nombre_categoria)

    # Filtrar por categoría
    subset = df[df["categoria_raw"] == key]

    # Conteos por split y etiqueta
    for split in ["Train", "Valid"]:
        subset_split = subset[subset["split"] == split]
        normales = subset_split[subset_split["etiqueta"] == 0].shape[0]
        anormales = subset_split[subset_split["etiqueta"] == 1].shape[0]
        conteo[f"Normal_{split}"].append(normales)
        conteo[f"Anormal_{split}"].append(anormales)

# Crear DataFrame
df_conteo = pd.DataFrame(conteo)

# Agregar fila de totales
totales = ["Total"]
for col in df_conteo.columns[1:]:
    totales.append(df_conteo[col].sum())
df_conteo.loc[len(df_conteo)] = totales

# Mostrar la tabla
print(df_conteo)

# Guardar si querés
df_conteo.to_csv(os.path.join(ruta_base, "distribucion_MURA_desde_csv.csv"), index=False)
# df_conteo.to_latex(index=False)  # si querés pegarlo en tu tesis


   Categoría  Normal_Train  Anormal_Train  Normal_Valid  Anormal_Valid
0       Codo          2559           1773           601            463
1       Dedo          2731           1778           621            437
2  Antebrazo          1038            664           276            148
3       Mano          3397           1347           933            326
4     Húmero           646            589           175            150
5     Hombro          3591           3541           905            905
6     Muñeca          4890           3460          1239            822
7      Total         18852          13152          4750           3251
